# RAG On JFK Speeches: Part 2


__[1. Introduction to RAG ](#first-bullet)__

__[2. Retriving Documents With Vector (Semantic) Search](#second-bullet)__

__[3. Building A RAG Pipeline](#third-bullet)__

<!-- __[4. A CI/CD Pipeline For RAG](#fourth-bullet)__ -->

__[4. Deploying A RAG Application](#fourth-bullet)__

__[5. Conclusions](fifth-bullet)__



### 1. Introduction to RAG <a class="anchor" id="first-bullet"></a>
------------------------------

In my [last post](http://michael-harmon.com/blog/ragjfk1.html) on RAG I discussed how to ingest President Kennedy's speeches into a [Pinecone](https://www.pinecone.io/) vector database and perform semantic search  using both Pinecone's API as well as using the [Langchain](https://www.langchain.com/) API. I used Pinecone for a vector database since its cloud based, fully managed and of course has a free tier. In this post I will expand upon my prior work and build out a [Retrivial Augmented Generation (RAG)](https://en.wikipedia.org/wiki/Retrieval-augmented_generation) pipeline using Langchain. I will deploy this as a [Streamlit](https://streamlit.io/) application to be able to answer questions on President Kennedy.

You may ask what is the point of RAG pipelines? Don't [Large Language Models (LLMs)](https://en.wikipedia.org/wiki/Large_language_model) know answers to everything? The answer is most LLMs take a long time to train and are often trained on data that is out of date when people begin to use the model. In order to incorporate more recent data into our LLM we could use fine-tuning, but this can still be time consuming and costly. The other option is to use [Retrivial Augmented Generation (RAG)](https://en.wikipedia.org/wiki/Retrieval-augmented_generation). RAG takes your original question and  "retrieves" documents from a vector database that are most most semantically related to your qeustion. RAG is able to do semantic search by converting the text in your question and the documents to a numerical vectors using an [embedding](https://developers.google.com/machine-learning/crash-course/embeddings). The closeness of the document vectors to the question vector (with resepect to a norm) measures the semantic similarity. The original question and the retrieved documents are incorporated into a prompt which is fed into the LLM where they are used as "context" to generate an answer. The entire process is depicted below,


<figure>
    <img src="images/rag-pipeline.png" alt>
    <figcaption>Source: https://python.langchain.com/docs/tutorials/rag/</figcaption>
</figure>


I'll note that building a RAG pipeline was actually much easier than I originally thought which is a testament to the power and simplicity of the Langchain framework! 

Let's get started! 

I'll start out with all the necessary imports:

In [ ]:
# LangChain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore

# Pinecone VectorDB
from pinecone import Pinecone
from pinecone import ServerlessSpec

import os

# API Keys
from dotenv import load_dotenv
load_dotenv()

/Users/mikeharmon/miniconda3/envs/llm_env/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

## 2. Retriving Documents With Vector (Semantic) Search <a class="anchor" id="second-bullet"></a>

First thing we'll do is review retrivial with semantic search again. This is important since I will dicuss a more useful way to interact with the Vector databse using a so-called "retrivier." This functionality will be particularly helpful for a RAG pipeline. 

The first thing I need to do is connect to the Pinecone database and make sure the index of vectors corresponding to President Kennedy's speches exists:

In [3]:
index_name = "prez-speeches"

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'prez-speeches-2307pwa.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'prez-speeches',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

Now that we have confirmed the index exists and is ready for querying we can create the initial connection to the Vector database using the Langchain [PineconeVectorStore](https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html) class. Note that we have to pass the name of the index as well as the embeddings to the class' constructor. It's important that we use the same embeddings here that we used to convert the speeches to numerical vectors in the Pinecone index.

In [4]:
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')

vectordb = PineconeVectorStore(
                    pinecone_api_key=os.getenv("PINECONE_API_KEY"),
                    embedding=embedding,
                    index_name=index_name
)

Now we can perform vector similarity search using the [similiarity search](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/example_selectors/similarity/) function in Langchain. Under the hood this function creates a vector embedding of your question (query) and finds the closest documents using the cosine similiarity score between the embedded question vector and the embedded document vectors. The determination of closest documents to the question are calculated by the "nearest neighbors" algorithm. This process is depicted in image below,


<figure>
    <img src="images/vector-search.jpg" alt>
    <figcaption>Source: https://www.elastic.co/what-is/vector-search</figcaption>
</figure>

The one thing to note is that I use the async similarity search for funsies and set it to return the top 5 documents.

In [5]:
question = "How did President Kennedy feel about the Berlin Wall?"

results = await vectordb.asimilarity_search(query=question, k=5)

I'll print out the document id's since the actual text for the top 5 will be too long for the screen.

In [6]:
for document in results:
    print("Document ID:", document.id)

Document ID: 64fc63a1-79fd-4b40-bf8c-09f0617b9f0f
Document ID: 0fa5431f-a374-429e-a622-a1ed1c2b0a21
Document ID: 121366d4-9f46-4f52-8e56-2523bf1c9c8f
Document ID: 2da0bf3a-9adc-4dd0-a697-117bc3f0d8b9
Document ID: 4df626ad-0034-45cb-8144-88a21576785d


Now that we understand how to use the vector database to perform "retrivial" using similairty search, let's create a chain that will allow us to query the database and generate a response from the LLM. This will form the basis of a so-called "RAG Pipeline."

## 3. Building A RAG Pipeline <a class="anchor" id="third-bullet"></a>
--------------------------------

Now we can use the vector database as a [retriever](https://python.langchain.com/docs/integrations/retrievers/) which is a special Langchain [Runnable](https://python.langchain.com/api_reference/core/runnables.html) object that takes in a string (query) and returns a list of Langchain [Documents](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html). This is depicted below,

<figure>
    <img src="images/retriever.png" alt>
    <figcaption>Source: https://python.langchain.com/docs/concepts/retrievers/</figcaption>
</figure>


We can see this in action,

In [7]:
retriever = vectordb.as_retriever()
print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


Now we can query the vector database using the `invoke` method of the retriever:

In [8]:
documents = retriever.invoke(input=question)

for document in documents:
    print("Document ID:", document.id)

Document ID: 64fc63a1-79fd-4b40-bf8c-09f0617b9f0f
Document ID: 0fa5431f-a374-429e-a622-a1ed1c2b0a21
Document ID: 121366d4-9f46-4f52-8e56-2523bf1c9c8f
Document ID: 2da0bf3a-9adc-4dd0-a697-117bc3f0d8b9


Now let's talk about our prompt for RAG pipeline. 

I used the classic [rlm/rag-prompt](https://smith.langchain.com/hub/rlm/rag-prompt) from [LangSmith](https://www.langchain.com/langsmith). I couldn't use the original one as the function [create_retrieval_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html) expects the human input to be a variable `input` while the original prompt has the input be `question`. The whole prompt is,

In [9]:
from langchain.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {input} 
Context: {context} 
Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input", "context"],
)

Now I'll give an example of how to use this prompt. I'll use the question from the user as well as the documents retrieved from Pinecone as context:

In [10]:
print(
    prompt.invoke({
        "input": question,
        "context": [document.id for document in documents]
    }).text
)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: How did President Kennedy feel about the Berlin Wall? 
Context: ['64fc63a1-79fd-4b40-bf8c-09f0617b9f0f', '0fa5431f-a374-429e-a622-a1ed1c2b0a21', '121366d4-9f46-4f52-8e56-2523bf1c9c8f', '2da0bf3a-9adc-4dd0-a697-117bc3f0d8b9'] 
Answer:



Note I only used the document ids as context in the prompt. This is because printing the actual Langchain Documents would be a lot of text for the screen. However, in a real RAG pipeline we would pass the actual documents to the LLM.

Now we'll move on to create our LLM [ChatModel](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html) as this object will be needed to write the response to our question.

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

The LLM will be used as the generative part of the RAG pipeline. 

The generative component in our RAG pipelien will be created by a function called [create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html). This function will return a Runnable object and we'll give this object the name `generative_chain`:

In [12]:
generate_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

We can see what makes up this composite Runnable and the components of the chain:

In [13]:
print(generate_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {input} \nContext: {context} \nAnswer:\n")
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x112b621d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x112b9a190>, root_client=<openai.OpenAI object at 0x1134c9990>, root_async_client=<openai.AsyncOpenAI object at 0x112b83c50>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser() kwa

Now we can call the chain using the `invoke` method and see the answer to our question. 

The chain takes in the prompt as input, passes it to the LLM and then the [StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) which will return a string from the LLM instead of the [AIMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html) (which is the usual return type of a ChatModel).

In [14]:
answer = generate_chain.invoke(
       {
        'context': documents,
        "input": question
      }
)

In [15]:
print(answer)

President Kennedy viewed the Berlin Wall as a significant symbol of the failures of the Communist system, stating that it was an offense against humanity that divided families and friends. He expressed pride in the resilience of the people of West Berlin and emphasized the importance of freedom and the right to make choices. Kennedy's speeches highlighted his commitment to supporting the people of Berlin and the broader struggle for freedom worldwide.


Now we can put this all together as a RAG chain by passing the Pinecone Vector database retriever and the generative chain to the [create_retrieval_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html). The retriever will take in the input question and perform similarity search and return the documents. These documents along with the input question will be passed to the `generate_chain` to return the answer output. 

The full RAG chain is below:

In [16]:
rag_chain = create_retrieval_chain(
                    retriever=retriever, 
                    combine_docs_chain=generate_chain)

The definition of the `rag_chain` is a bit different from `generate_chain` above and we can see its compontents,

In [17]:
print(rag_chain)

bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x113490e50>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {input} \nContext: {con

We can see prompts that make up this chain:

In [18]:
rag_chain.get_prompts()

[PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'),
 PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {input} \nContext: {context} \nAnswer:\n")]

And then test it out,

In [19]:
response = rag_chain.invoke({"input": question})

In [22]:
response['answer']

"President Kennedy viewed the Berlin Wall as a significant symbol of the failures of the Communist system, stating that it was an offense against humanity that separated families and divided people. He expressed pride in the resilience of West Berlin and emphasized the importance of freedom and the right to make free choices. Kennedy's speeches reflected a strong commitment to supporting the people of Berlin and opposing the oppressive nature of communism."

The response will be a dictionary will look like,

    {
     'input': -> Input question
     'answer' -> LLM answer
     'context': -> List of documents
    }
    
and contains the input question and the answer generated by the model. It also includes the context for which are all documents that were the most semantically related to our question and passed to the LLM to use to generate an answer.

We can see the associated data with context reference documents which will be important for our deployment.

In [21]:
references = [(doc.metadata["title"], doc.metadata["url"]) for doc in response['context']]

references

[('Radio and Television Report to the American People on the Berlin Crisis, July 25, 1961',
  'https://www.jfklibrary.org//archives/other-resources/john-f-kennedy-speeches/berlin-crisis-19610725'),
 ('Remarks of President John F. Kennedy at the Rudolph Wilde Platz, Berlin, June 26, 1963',
  'https://www.jfklibrary.org//archives/other-resources/john-f-kennedy-speeches/berlin-w-germany-rudolph-wilde-platz-19630626'),
 ('Remarks of President John F. Kennedy at the Rudolph Wilde Platz, Berlin, June 26, 1963',
  'https://www.jfklibrary.org//archives/other-resources/john-f-kennedy-speeches/berlin-w-germany-rudolph-wilde-platz-19630626'),
 ('Address before the American Society of Newspaper Editors, Washington, D.C., April 20, 1961',
  'https://www.jfklibrary.org//archives/other-resources/john-f-kennedy-speeches/american-society-of-newspaper-editors-19610420')]

## 4. Deploying A RAG Application <a class="anchor" id="fourth-bullet"></a>
-------------------
Now in order to deploy this in a [Streamlit App](https://streamlit.io/) I'll create a function that called [ask_question](https://github.com/mdh266/rag-jfk/blob/main/app/rag.py) that takes in a `question` and an `index_name` for the vector database, it then runs all the logic we went through above and returns the response dictionary. I'll then print the answer from the LLM and then print out the retrieved documents as sources for the with the title as the speech and the the url as a hyperlink. The entire streamlit app with an example is shown below,

<center>
<img src="images/ragui.png" 
     width="600" 
     height="600"
     class="center" />
</center>

I won't go through the process of deploying this app to [Google Cloud Run](https://cloud.google.com/run?hl=en) as I have covered that pretty extensively in a [prior post](http://michael-harmon.com/blog/chatbot2.html).

## 5. Conclusions  <a class="anchor" id="fifth-bullet"></a>
-------------

In this post I covered the basics of creating a Retrivial Augumented Generation (RAG) App using Langchain and deploying it as a Streamlit App. The RAG application is based on Speeches made by President Kenendy and were stored in a Pinecone Vector database. In a future post I will go over methods of evaluating and testing the RAG pipeline, but this is enough for now. Hope you enjoyed it!